<a href="https://colab.research.google.com/github/Wnjoki/PILyTQOJRh9c4klH/blob/main/Potential_Talent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [1]:
# data processing
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import string

#Text-Processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import re

#models
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch






[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Data Pre-Processing**

In [2]:
#load the data
talent_df = pd.read_csv("/content/potential-talents - Aspiring human resources - seeking human resources.csv")

In [3]:
talent_df.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [4]:
talent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [5]:
#talent_df['job_title'].unique()

In [6]:
Stopwords = set(stopwords.words('english'))
def pre_process(text):
    text = str(text)
    text = text.lower() #Converting to lowercase
    #removing punctuation
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ',text)
    text = re.sub('<.*?>+',' ',text) #removing HTML Tags
    text = re.sub('[^a-zA-Z0-9\n]', ' ', text) #replacing special character with space
    text = re.sub(r'\s+', ' ',text) #removal of multiple spaces

    text_tokens = word_tokenize(text) #tokenization

    #stemming
    stemmer = PorterStemmer()
    text_tokens = [stemmer.stem(token) for token in text_tokens]

    #removing stopwords
    tw = [word for word in text_tokens if word not in Stopwords]
    text = (" ").join(tw)
    return text

In [7]:
talent_df['job_title'] = talent_df['job_title'].apply(pre_process)

In [8]:
talent_df.head()

,id,job_title,location,connection,fit
0,1,2019 c bauer colleg busi graduat magna cum lau...,"Houston, Texas",85,NaN
1,2,nativ english teacher epik english program korea,Kanada,500+,NaN
2,3,aspir human resourc profession,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,peopl develop coordin ryan,"Denton, Texas",500+,NaN
4,5,advisori board member celal bayar univers,"İzmir, Türkiye",500+,NaN


In [9]:
plot_dist = talent_df['connection'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all connections',text_auto=True)
fig.show()

500+ have the highest values

In [10]:
plot_dist = talent_df['location'] .value_counts().sort_index()
fig = px.bar(plot_dist,title='Frequency Distribution for all locations',text_auto=True)
fig.show()

Kanada has the highest number of candidates.

#Model Training

In [11]:
#create a df with relevant columns
talent_df2 = talent_df[['job_title', 'connection']]
talent_df2.head()

,job_title,connection
0,2019 c bauer colleg busi graduat magna cum lau...,85
1,nativ english teacher epik english program korea,500+
2,aspir human resourc profession,44
3,peopl develop coordin ryan,500+
4,advisori board member celal bayar univers,500+


doc2vec embeddings

In [12]:
# Doc2Vec Embeddings
tagged_data = [TaggedDocument(words=title.split(), tags=[str(i)]) for i, title in enumerate(talent_df2['job_title'])]


In [13]:
# Train Doc2Vec model
model = Doc2Vec(tagged_data, vector_size=100, window=2, min_count=1, workers=4, epochs=20)


In [14]:
# Doc2Vec embedding for "Aspiring human resources" (w1)
w1 = "Aspiring human resources"
w1_vector = model.infer_vector(w1.split())

In [15]:
#  cosine similarity between w1 and job_title embeddings
talent_df2.loc[:, 'cosine_similarity'] = np.nan  # Initialize the cosine_similarity column with NaN values

for i, title in enumerate(talent_df2['job_title']):
    title_words = title.split()
    similarity = cosine_similarity([model.infer_vector(title_words)], [w1_vector])[0][0]
    talent_df2.loc[i, 'cosine_similarity'] = similarity

<ipython-input-15-72433e54af5d>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [16]:
# Convert "connection" column to numeric format
talent_df2.loc[:, 'connection'] = talent_df2.loc[:, 'connection'].str.replace('+', '').astype(float)


<ipython-input-16-3f1ece20f6e2>:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-16-3f1ece20f6e2>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-16-3f1ece20f6e2>:2: DeprecationWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



In [17]:
# Scale "connection" column between 0 and 1
scaler = MinMaxScaler()
talent_df2.loc[:, 'scaled_connection'] = scaler.fit_transform(talent_df2[['connection']])


<ipython-input-17-676a83335dcc>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
talent_df2

,job_title,connection,cosine_similarity,scaled_connection
0,2019 c bauer colleg busi graduat magna cum lau...,85.0,0.129488,0.168337
1,nativ english teacher epik english program korea,500.0,0.012343,1.000000
2,aspir human resourc profession,44.0,-0.172661,0.086172
3,peopl develop coordin ryan,500.0,-0.009478,1.000000
4,advisori board member celal bayar univers,500.0,0.011742,1.000000
...,...,...,...,...
99,aspir human resourc manag graduat may 2020 see...,103.0,0.160432,0.204409
100,human resourc generalist loparex,500.0,0.119455,1.000000
101,busi intellig analyt travel,49.0,0.160952,0.096192
102,alway set success,500.0,0.211930,1.000000


In [19]:
# Calculating ranking
talent_df2['ranking'] = 0.8 * talent_df2['cosine_similarity'] + 0.2 * talent_df2['scaled_connection']


In [20]:
talent_df2

,job_title,connection,cosine_similarity,scaled_connection,ranking
0,2019 c bauer colleg busi graduat magna cum lau...,85.0,0.129488,0.168337,0.137258
1,nativ english teacher epik english program korea,500.0,0.012343,1.000000,0.209875
2,aspir human resourc profession,44.0,-0.172661,0.086172,-0.120894
3,peopl develop coordin ryan,500.0,-0.009478,1.000000,0.192418
4,advisori board member celal bayar univers,500.0,0.011742,1.000000,0.209393
...,...,...,...,...,...
99,aspir human resourc manag graduat may 2020 see...,103.0,0.160432,0.204409,0.169227
100,human resourc generalist loparex,500.0,0.119455,1.000000,0.295564
101,busi intellig analyt travel,49.0,0.160952,0.096192,0.148000
102,alway set success,500.0,0.211930,1.000000,0.369544


import numpy as np
from google.colab import autoviz
df_8539504025764446430 = autoviz.get_registered_df('df_8539504025764446430')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_8539504025764446430, *['connection'], **{})
chart

import numpy as np
from google.colab import autoviz
df_7336283511217361012 = autoviz.get_registered_df('df_7336283511217361012')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_7336283511217361012, *['cosine_similarity'], **{})
chart

import numpy as np
from google.colab import autoviz
df_2112267582816794566 = autoviz.get_registered_df('df_2112267582816794566')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_2112267582816794566, *['scaled_connection'], **{})
chart

import numpy as np
from google.colab import autoviz
df_856716887400741505 = autoviz.get_registered_df('df_856716887400741505')

def value_plot(df, y, sort_ascending=False, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  if sort_ascending:
    df = df.sort_values(y).reset_index(drop=True)
  _, ax = plt.subplots(figsize=figsize)
  df[y].plot(kind='line')
  plt.title(y)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(df_856716887400741505, *['ranking'], **{})
chart

import numpy as np
from google.colab import autoviz
df_3978868809721727827 = autoviz.get_registered_df('df_3978868809721727827')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_3978868809721727827, *['connection'], **{})
chart

import numpy as np
from google.colab import autoviz
df_841425053033063725 = autoviz.get_registered_df('df_841425053033063725')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_841425053033063725, *['cosine_similarity'], **{})
chart

import numpy as np
from google.colab import autoviz
df_449735718910002138 = autoviz.get_registered_df('df_449735718910002138')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_449735718910002138, *['scaled_connection'], **{})
chart

import numpy as np
from google.colab import autoviz
df_2786653834875871265 = autoviz.get_registered_df('df_2786653834875871265')

def histogram(df, colname, num_bins=20, figsize=(2, 1)):
  from matplotlib import pyplot as plt
  _, ax = plt.subplots(figsize=figsize)
  plt.hist(df[colname], bins=num_bins, histtype='stepfilled')
  plt.ylabel('count')
  plt.title(colname)
  ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(df_2786653834875871265, *['ranking'], **{})
chart

import numpy as np
from google.colab import autoviz
df_6967555103049481302 = autoviz.get_registered_df('df_6967555103049481302')

def scatter_plots(df, colname_pairs, scatter_plot_size=2.5, size=8, alpha=.6):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * scatter_plot_size, scatter_plot_size))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    ax.scatter(df[x_colname], df[y_colname], s=size, alpha=alpha)
    plt.xlabel(x_colname)
    plt.ylabel(y_colname)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(df_6967555103049481302, *[[['connection', 'cosine_similarity'], ['cosine_similarity', 'scaled_connection'], ['scaled_connection', 'ranking']]], **{})
chart

Bert Embeddings

In [21]:
# BERT Embeddings
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
#generate BERT embeddings
def get_bert_embedding(text):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)
    outputs = model_bert(input_ids)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0).detach().numpy()
    return embeddings

In [23]:
# BERT embedding for "Aspiring human resources" (w1)
w1_embedding = get_bert_embedding(w1)

print(w1_embedding[:3])

[ 0.30581912 -0.08324541  0.24562995]


In [24]:
# Cosine similarity between w1 and job_title BERT embeddings
talent_df2['bert_embedding'] = talent_df2['job_title'].apply(get_bert_embedding)
talent_df2['cosine_similarity'] = talent_df2['bert_embedding'].apply(lambda x: cosine_similarity([x], [w1_embedding])[0][0])

In [25]:
# Scale "connection" column between 0 and 1
scaler = MinMaxScaler()
talent_df2['scaled_connection'] = scaler.fit_transform(talent_df2[['connection']])


In [26]:
# Ranking based on weighted sum of cosine_similarity and scaled_connection
talent_df2['ranking'] = 0.8 * talent_df2['cosine_similarity'] + 0.2 * talent_df2['scaled_connection']


In [27]:
# Sort the dataframe
talent_df2 = talent_df2.sort_values(by='ranking', ascending=False)

talent_df2.head()

,job_title,connection,cosine_similarity,scaled_connection,ranking,bert_embedding
7,hr senior specialist,500.0,0.80481,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
37,hr senior specialist,500.0,0.80481,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
60,hr senior specialist,500.0,0.80481,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
25,hr senior specialist,500.0,0.80481,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
50,hr senior specialist,500.0,0.80481,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."


In [28]:
#Top  candidates with highest ranking scores
top_candidates = talent_df2.head(7)

In [29]:
# Print the top candidates
top_candidates

,job_title,connection,cosine_similarity,scaled_connection,ranking,bert_embedding
7,hr senior specialist,500.0,0.804810,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
37,hr senior specialist,500.0,0.804810,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
60,hr senior specialist,500.0,0.804810,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
25,hr senior specialist,500.0,0.804810,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
50,hr senior specialist,500.0,0.804810,1.0,0.843848,"[-0.06280377, -0.15042314, 0.1989594, -0.23628..."
66,human resourc staf recruit profession,500.0,0.672640,1.0,0.738112,"[0.44099912, 0.039281614, -0.20177273, 0.10743..."
26,aspir human resourc manag student seek internship,500.0,0.637012,1.0,0.709610,"[0.0015275904, 0.07670096, 0.0019320186, 0.097..."
